# Visualização

In [0]:
%sql
USE default;
SELECT * FROM pipeline_delta;

data_publicacao,titulo,autor,descricao,url,fonte,conteudo
2024-09-25,"Singular Genomics Systems, Inc. (NASDAQ:OMIC) Short Interest Up 830.4% in September",MarketBeat News,"Singular Genomics Systems, Inc. (NASDAQ:OMIC – Get Free Report) was the target of a significant increase in short interest in September. As of September 15th, there was short interest totalling 107,000 shares, an increase of 830.4% from the August 31st total …",https://www.etfdailynews.com/2024/09/25/singular-genomics-systems-inc-nasdaqomic-short-interest-up-830-4-in-september/,ETF Daily News,"Singular Genomics Systems, Inc. (NASDAQ:OMIC – Get Free Report) was the target of a significant increase in short interest in September. As of September 15th, there was short interest totalling 107,0… [+2678 chars]"
2024-10-22,Telo Genomics Presents New Performance Data for Its Smoldering Myeloma Prognostic Test,Investing.com,Telo Genomics Presents New Performance Data for Its Smoldering Myeloma Prognostic Test,https://www.investing.com/news/press-releases/telo-genomics-presents-new-performance-data-for-its-smoldering-myeloma-prognostic-test-93CH-3675681,Investing.com,"Toronto, Ontario--(Newsfile Corp. - October 22, 2024) - Telo Genomics Corp. (TSXV: TELO) (OTCQB: TDSGF) (the ""Company"" or ""Telo"") a biotech company developing the industry's leading telomere technolo… [+6366 chars]"
2024-09-27,"Public Employees Retirement Association of Colorado Sells 325 Shares of Bio-Rad Laboratories, Inc. (NYSE:BIO)",MarketBeat News,"Public Employees Retirement Association of Colorado trimmed its position in Bio-Rad Laboratories, Inc. (NYSE:BIO – Free Report) by 13.5% during the second quarter, HoldingsChannel reports. The institutional investor owned 2,086 shares of the medical research …",https://www.etfdailynews.com/2024/09/27/public-employees-retirement-association-of-colorado-sells-325-shares-of-bio-rad-laboratories-inc-nysebio/,ETF Daily News,"Public Employees Retirement Association of Colorado trimmed its position in Bio-Rad Laboratories, Inc. (NYSE:BIO – Free Report) by 13.5% during the second quarter, HoldingsChannel reports. The instit… [+6190 chars]"
2024-09-30,"Wellington Wealth Strategies LLC. Buys Shares of 1,782 Agilent Technologies, Inc. (NYSE:A)",MarketBeat News,"Wellington Wealth Strategies LLC. purchased a new position in shares of Agilent Technologies, Inc. (NYSE:A – Free Report) during the 2nd quarter, according to its most recent 13F filing with the Securities and Exchange Commission (SEC). The fund purchased 1,7…",https://www.etfdailynews.com/2024/09/30/wellington-wealth-strategies-llc-buys-shares-of-1782-agilent-technologies-inc-nysea/,ETF Daily News,"Wellington Wealth Strategies LLC. purchased a new position in shares of Agilent Technologies, Inc. (NYSE:A – Free Report) during the 2nd quarter, according to its most recent 13F filing with the Secu… [+7078 chars]"
2024-09-23,Clinical Trials Market to Witness 8.7% CAGR by 2031 | SkyQuest Technology,SkyQuest Technology Consulting Pvt. Ltd.,"Clinical Trials Market size was valued at USD 52.39 billion in 2023 to USD 102.12 billion by 2031, growing at a CAGR of 8.7% in the forecast period (2024-2031). Clinical Trials Market size was valued at USD 52.39 billion in 2023 to USD 102.12 billion by 2031,…",https://www.globenewswire.com/news-release/2024/09/23/2951383/0/en/Clinical-Trials-Market-to-Witness-8-7-CAGR-by-2031-SkyQuest-Technology.html,GlobeNewswire,"Westford, USA, Sept. 23, 2024 (GLOBE NEWSWIRE) -- SkyQuest projects that Clinical Trials Market will attain the value of USD 102.12 Billion by 2031, with a CAGR of 8.7% during the forecast period (… [+8579 chars]"
2024-10-20,Genética a la carta: la nueva frontera de la fecundación 'in vitro' suscita controversia,"RT en Español , RT en Español","Una nueva y controvertida tecnología permite analizar el cociente intelectual de los embriones, además de otras características que pueden ser preferibles para los futuros padres.",https://actualidad.rt.com/actualidad/527

In [0]:
%sql
USE default;
SELECT * FROM artigos_ano_mes_dia;

year,month,day,count
2024,10,10,39
2024,9,25,29
2024,9,19,29
2024,9,26,27
2024,9,23,25
2024,9,18,25
2024,10,16,25
2024,10,11,23
2024,9,20,22
2024,9,17,22


In [0]:
%sql
USE default;
SELECT * FROM artigos_fonte_autor;

fonte,autor,count
ETF Daily News,MarketBeat News,193
GlobeNewswire,Research and Markets,41
Investing.com,Investing.com,24
Science Daily,null,20
Biztoc.com,marketbeat.com,18
Investor's Business Daily,Investor's Business Daily,9
National Institutes of Health,null,7
Financial Post,GlobeNewswire,5
Cdc.gov,null,5
[Removed],null,5


In [0]:
%sql
USE default;
SELECT * FROM artigos_palavras_chave;

year,month,sum_DNA,sum_genetic,sum_treatment
2024,9,12,9,1
2024,10,11,17,5
